https://mlflow.org/docs/latest/deep-learning/tensorflow/guide/index.html

In [1]:
import mlflow
import numpy as np
import tensorflow as tf
from tensorflow import keras

2024-06-26 15:44:39.719703: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


logging system metrics

In [2]:
import os

os.environ["MLFLOW_ENABLE_SYSTEM_METRICS_LOGGING"] = "true"

defining tracking server

In [3]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("my-experiment")
mlflow.enable_system_metrics_logging()

logging training

In [4]:
mlflow.tensorflow.autolog()

# Prepare data for a 2-class classification.
data = np.random.uniform(size=[20, 28, 28, 3])
label = np.random.randint(2, size=20)

model = keras.Sequential(
    [
        keras.Input([28, 28, 3]),
        keras.layers.Conv2D(8, 2),
        keras.layers.MaxPool2D(2),
        keras.layers.Flatten(),
        keras.layers.Dense(2),
        keras.layers.Softmax(),
    ]
)

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(0.001),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

with mlflow.start_run() as run:
    model.fit(data, label, batch_size=5, epochs=5)

2024-06-26 15:44:41.898571: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2024/06/26 15:44:42 WARNING mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics because creating `GPUMonitor` failed with error: `pynvml` is not installed, to log GPU metrics please run `pip install pynvml` to install it..
2024/06/26 15:44:42 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.


Epoch 1/5
2/4 [==============>...............] - ETA: 0s - loss: 0.7578 - sparse_categorical_accuracy: 0.6000

2024/06/26 15:44:43 WARNING mlflow.utils.checkpoint_utils: Checkpoint logging is skipped, because checkpoint 'save_best_only' config is True, it requires to compare the monitored metric value, but the provided monitored metric value is not available.


4/4 [==============================] - 1s 39ms/step - loss: 0.8343 - sparse_categorical_accuracy: 0.5500
Epoch 2/5
3/4 [=====================>........] - ETA: 0s - loss: 0.6635 - sparse_categorical_accuracy: 0.6000

2024/06/26 15:44:43 WARNING mlflow.utils.checkpoint_utils: Checkpoint logging is skipped, because checkpoint 'save_best_only' config is True, it requires to compare the monitored metric value, but the provided monitored metric value is not available.


...

4/4 [==============================] - 0s 34ms/step - loss: 0.6947 - sparse_categorical_accuracy: 0.5500
Epoch 5/5
4/4 [==============================] - ETA: 0s - loss: 0.6356 - sparse_categorical_accuracy: 0.6500

2024/06/26 15:44:43 WARNING mlflow.utils.checkpoint_utils: Checkpoint logging is skipped, because checkpoint 'save_best_only' config is True, it requires to compare the monitored metric value, but the provided monitored metric value is not available.


1/1 [==============================] - 0s 87ms/step


INFO:tensorflow:Assets written to: /tmp/tmp0trw01pw/model/data/model/assets


INFO:tensorflow:Assets written to: /tmp/tmp0trw01pw/model/data/model/assets
2024/06/26 15:44:49 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/thiagoads/Workspace/Programas/miniconda3/envs/tensorflow-mlflow/lib/python3.9/site-packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutils."
2024/06/26 15:44:50 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2024/06/26 15:44:50 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!


loading logged model

In [13]:
logged_model = f'runs:/{run.info.run_id}/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

loaded_model.predict(data)

1/1 [==============================] - 0s 60ms/step


array([[0.5366621 , 0.4633379 ],
       [0.5842944 , 0.4157056 ],
       [0.5008931 , 0.4991069 ],
       [0.64047194, 0.35952812],
       [0.46039307, 0.539607  ],
       [0.62187225, 0.37812775],
       [0.53660834, 0.46339172],
       [0.6308145 , 0.36918548],
       [0.6058376 , 0.39416236],
       [0.48654717, 0.5134528 ],
       [0.44766453, 0.55233544],
       [0.41407704, 0.58592296],
       [0.6876836 , 0.31231642],
       [0.68955415, 0.31044587],
       [0.64791745, 0.35208258],
       [0.62317276, 0.3768273 ],
       [0.6011608 , 0.39883912],
       [0.59386677, 0.4061332 ],
       [0.44534793, 0.55465204],
       [0.5793105 , 0.42068955]], dtype=float32)

serving model as api (+ https://mlflow.org/docs/latest/deployment/deploy-model-locally.html)

In [35]:
model_serve_cmd = f"MLFLOW_TRACKING_URI=http://127.0.0.1:5000 mlflow models serve -m runs:/{run.info.run_id}/model --port 1234 --no-conda"
print("Copy and Paste the instruction below in terminal to serve model as API: \n")
print(model_serve_cmd)

Copy and Paste the instruction below in terminal to serve model as API: 

MLFLOW_TRACKING_URI=http://127.0.0.1:5000 mlflow models serve -m runs:/128a79d01cd548208ab33779a70a085b/model --port 1234 --no-conda


saving data for test throught api

In [54]:
import json
from numpyencoder import NumpyEncoder

# saving train data
with open('data/train.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, cls=NumpyEncoder)

In [55]:
# generating test data
test = json.dumps(np.random.uniform(size=[1, 28, 28, 3]), cls=NumpyEncoder)
print(test)
with open('data/test.json', 'w', encoding='utf-8') as f:
    json.dump(test, f, cls=NumpyEncoder)

[[[[0.2536740283889829, 0.5607512350211892, 0.41024849343795866], [0.2342227819692806, 0.33891873171396636, 0.08577671082789984], [0.8520130183998889, 0.09259084788524496, 0.9042020000369053], [0.06861858877316174, 0.2266556028386726, 0.9283107083598898], [0.339266044190757, 0.6594547036551582, 0.19036035858699973], [0.8710566187822467, 0.8596697962211551, 0.5655815986976421], [0.8694115501870241, 0.5561378633332343, 0.3206334928654495], [0.9267093298028851, 0.7477239799421007, 0.8858284375967892], [0.8484893828182528, 0.42553398963784106, 0.9481833911204938], [0.08877736500813427, 0.04269230986783601, 0.3801205220325623], [0.6171822414674482, 0.219746410124392, 0.005283649712445215], [0.8887613627665939, 0.24871369419180211, 0.711134995597327], [0.6173591871296712, 0.5994527908370784, 0.763697975903505], [0.9469116493047574, 0.8745472670656864, 0.3271852255432588], [0.33251693580589503, 0.17619233959635816, 0.47041427750597686], [0.9155019602711366, 0.14020969188983334, 0.974955158156

calling the api

In [56]:
curl_cmd = "curl -i -X POST -H 'Content-type: application/json' --data @data/payload.json http://127.0.0.1:1234/invocations" 
print(curl_cmd)

curl -i -X POST -H 'Content-type: application/json' --data @data/payload.json http://127.0.0.1:1234/invocations


In [59]:
!$curl_cmd

HTTP/1.1 200 OK
Server: gunicorn
Date: Wed, 26 Jun 2024 20:42:34 GMT
Connection: close
Content-Type: application/json
Content-Length: 60

{"predictions": [[0.6007983088493347, 0.39920172095298767]]}